# DESaster Simulation Set Up Template

## Required Modules

In [287]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy import Resource, Container, FilterStore
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, io, movement, search, rebuild

## Input Data

In [288]:
scenario_file = '../inputs/scenario_test_renters.xlsx'

In [289]:
# Create Pandas dataframe of attribute data for all owners to be modeled in the simulation
owners_df = pd.read_excel(scenario_file, sheetname='owners')
owners_df

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Alfred,30000,10000,0,62 That St,Mobile Home,100,1,1,700,1920,100000,None,Own,Own,Single Family Dwelling
1,Bruce,100000,1000000,10000000,720 This Rd,Single Family Dwelling,100000,5,5,5000,1920,10000000,Slight,Own,Own,Single Family Dwelling
2,Selena,10000,100,0,1001 Other Ave,Single Family Dwelling,10,0,1,250,1960,10000,Complete,Own,Own,Single Family Dwelling
3,Fish,50000,1000,550000,26000 Out There Lane,Single Family Dwelling,2000,4,2,2000,2010,800000,Moderate,Own,Own,Single Family Dwelling


In [290]:
# Create Pandas dataframe of attribute data for all renters to be modeled in the simulation
renters_df = pd.read_excel(scenario_file, sheetname='renters')
renters_df

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Ivy,30000,10000,0,262 That St,Mobile Home,100,1,1,700,1920,100000,None,Rent,Rent,Single Family Dwelling
1,Edward,100000,50000,10000000,4720 This Rd,Single Family Dwelling,100000,3,2,5000,1920,10000000,Slight,Own,Rent,Single Family Dwelling
2,Oswald,10000,100,0,2301 Other Ave,Multi Family Dwelling,10,0,1,250,1960,10000,Complete,Rent,Rent,Multi Family Dwelling
3,James,50000,1000,550000,74000 Out There Lane,Single Family Dwelling,2000,2,2,2000,2010,800000,Moderate,Own,Rent,Single Family Dwelling


In [291]:
# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]
human_cap_data

Type
Contractors             40
Engineers               20
FEMA Processors         40
Inspectors               4
Insurance Adjusters     40
Loan Processors         40
Permit Processors      100
Name: Quantity, dtype: int64

In [292]:
# Set input data for all financial capital types, as dict or Pandas Series
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]
financial_cap_data

Type
Building Materials     2000000
FEMA Aid              35000000
Name: Quantity, dtype: int64

In [293]:
# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
forsale_stock_df = pd.read_excel(scenario_file, sheetname='forsale_stock')
forsale_stock_df.head()

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 New Ave,Mobile Home,100,1,1,700,1920,99999,Complete
1,101 New Ave,Single Family Dwelling,100000,6,5,5000,1920,9999,Complete
2,102 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,Complete
3,103 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,Complete
4,104 New Ave,Mobile Home,100,1,1,700,1920,100000,Complete


In [294]:
# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
forrent_stock_df = pd.read_excel(scenario_file, sheetname='forrent_stock')
forrent_stock_df.head()

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 Old Ave,Multi Family Dwelling,100,1,1,700,1920,99999,Slight
1,101 Old Ave,Single Family Dwelling,100000,2,2,5000,1920,9999,Slight
2,102 Old Ave,Multi Family Dwelling,10,0,1,250,1960,9999,Slight
3,103 Old Ave,Single Family Dwelling,2000,3,2,2000,2010,800000,Slight
4,104 Old Ave,Multi Family Dwelling,100,1,1,700,1920,100000,Slight


## Simulation Initiation

__Set simulation environment__

In [295]:
simulation = simpy.Environment()

__Indicate whether want to keep track of the stories of each entity (household) in the simulation.__

In [296]:
write_story = True

__Populate the simulation with human and financial capital data.__

In [297]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

human_capital = capitals.HumanCapital(simulation, human_cap_data)

__Create and populate a FilterStore as a vacant housing stock.__

In [298]:
forsale_stock = capitals.importHousingStock(simulation, forsale_stock_df)

__Create an empty FilterStore to use as the occupied housing stock.__

In [299]:
owned_stock = FilterStore(simulation)
rented_stock = FilterStore(simulation)

__Import data on households. Everytime a household's residence is created put it in the occupied housing stock.__

In [300]:
owners = entities.importOwners(simulation, owned_stock, owners_df, write_story)

In [301]:
renters = entities.importRenters(simulation, rented_stock, renters_df, write_story)

In [302]:
landlords = []

for renter in renters:
    landlords.append(entities.Landlord())

In [303]:
entities.assignLandlords(renters, landlords, write_story)

In [304]:
renters[1] == renters[1].landlord.tenant

True

In [305]:
 renters[1].residence == renters[1].landlord.residence

True

__Write a master process that combines process and functions from search, rebuild, and request modules.__

In [306]:
def landlord_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 365  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
            entity.residence = []
            entity.tenant.residence = []
            return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, forsale_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                entity.residence = []
                entity.tenant.residence = []
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
               
   
   

In [307]:
def owner_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 365  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
                return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, forsale_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
               
   

In [308]:
def renter_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(landlord_process(simulation, human_capital, financial_capital, entity.landlord, write_story))

In [309]:
# Initiate a master process for each owner to be modeled in the simulation
for i in range(len(owners)):
    simulation.process(owner_process(simulation, human_capital, financial_capital, owners[i], write_story))

In [310]:
# Initiate a master process for each renter to be modeled in the simulation
for i in range(len(renters)):
    simulation.process(renter_process(simulation, human_capital, financial_capital, renters[i], write_story))

__Can do some cool stuff with the vacant housing stock.__

In [311]:
# Do inspections on all of the vacant homes in the housing stock
for home in forsale_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

# Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# with probability = fix_probability
fix_probability = 1.0
fix_schedule = 100

# start_delayed(simulation, rebuild.stock(simulation, housing_stock, fix_probability), fix_schedule)

In [312]:
simulation.run()

## Outputs

__Owner summary statistics__

In [313]:
num_damaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_homesearch = 0
num_gave_up_home_search = 0

for household in owners:
    if household.money_search_start > 0.0: num_damaged += 1
    if household.home_get > 0.0: num_rebuilt += 1
    if household.gave_up_money_search: num_gave_up_money_search += 1
    if household.home_search_start > 0.0: num_homesearch += 1
    if household.home_search_stop > 0.0: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
        
print('{0} out of {1} owners suffered damage to their homes.\n'.format(num_damaged, len(owners)),
      '{0} out of {1} owners rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(owners)),
        '{0} out of {1} owners gave up searching for money.\n'.format(num_gave_up_money_search, len(owners)),
      '{0} out of {1} owners searched for a new home.\n'.format(num_homesearch, len(owners)),
        '{0} out of {1} owners bought a new home.\n'.format(num_relocated, len(owners)),
        '{0} out of {1} owners gave up searching for a home.'.format(num_gave_up_home_search, len(owners))
      )

3 out of 4 owners suffered damage to their homes.
 2 out of 4 owners rebuilt or repaired their damaged home.
 0 out of 4 owners gave up searching for money.
 1 out of 4 owners searched for a new home.
 0 out of 4 owners bought a new home.
 1 out of 4 owners gave up searching for a home.


__Owner stories__

In [314]:
owners[0].story

['Alfred owns and lives in a 1 bedroom Mobile Home at 62 That St worth $100,000. ',
 "Alfred's house was inspected 50 days after the event and suffered $0 of damage."]

In [315]:
owners[1].story

['Bruce owns and lives in a 5 bedroom Single Family Dwelling at 720 This Rd worth $10,000,000. ',
 "Bruce's house was inspected 50 days after the event and suffered $200,000 of damage.",
 'Bruce submitted an insurance claim 50 days after the event. ',
 'Bruce received a $200,000 insurance payout 65 days after the event. ',
 'It took Bruce 15 days to receive enough financial assistance and now has $1,200,000 to rebuild. ',
 'Bruce received an engineering assessment 90 days after the event. ',
 'Bruce received permit approval 125 days after the event. ',
 "Bruce's home was repaired 127 days after the event, taking 2 days to repair. "]

In [316]:
owners[2].story

['Selena owns and lives in a 0 bedroom Single Family Dwelling at 1001 Other Ave worth $10,000. ',
 "Selena's house was inspected 50 days after the event and suffered $10,000 of damage.",
 'Selena submitted a request to FEMA 50 days after the event. ',
 'Selena received $10,000 from FEMA 70 days after the event. ',
 'It took Selena 20 days to receive enough financial assistance and now has $10,100 to rebuild. ',
 'Selena started searching for a Single Family Dwelling with a value under $10,000 70 days after the event. ',
 'On day 620, after a 550 day search, Selena gave up looking for a new home in the local area. ']

In [317]:
owners[3].story

['Fish owns and lives in a 4 bedroom Single Family Dwelling at 26000 Out There Lane worth $800,000. ',
 "Fish's house was inspected 50 days after the event and suffered $80,000 of damage.",
 'Fish submitted an insurance claim 50 days after the event. ',
 'Fish received a $80,000 insurance payout 65 days after the event. ',
 'It took Fish 15 days to receive enough financial assistance and now has $81,000 to rebuild. ',
 'Fish received an engineering assessment 90 days after the event. ',
 'Fish received permit approval 125 days after the event. ',
 "Fish's home was repaired 155 days after the event, taking 30 days to repair. "]

__Renter summary statistics__

In [318]:
num_damaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_displaced = 0
num_gave_up_home_search = 0

for renter in renters:

    if renter.landlord.money_search_start > 0.0: num_damaged += 1
    if renter.landlord.home_get > 0.0: num_rebuilt += 1
    if renter.landlord.gave_up_money_search: num_gave_up_money_search += 1
    if not renter.residence: num_displaced += 1
        
print('{0} out of {1} renters\' homes suffered damage.\n'.format(num_damaged, len(renters)),
      '{0} out of {1} renters\' damaged home was rebuilt or repaired.\n'.format(num_rebuilt, len(renters)),
      '{0} out of {1} renters\' were displaced.\n'.format(num_displaced, len(renters)),
      )

3 out of 4 renters' homes suffered damage.
 2 out of 4 renters' damaged home was rebuilt or repaired.
 1 out of 4 renters' were displaced.



__Renters Stories__

In [319]:
renters[0].story + renters[0].landlord.story

['Ivy rents and lives in a 1 bedroom Mobile Home at 262 That St worth $100,000. ',
 "Ivy's residence is owned by Landlord 210. ",
 "Landlord 210's house was inspected 60 days after the event and suffered $0 of damage."]

In [320]:
renters[1].story + renters[1].landlord.story

['Edward rents and lives in a 3 bedroom Single Family Dwelling at 4720 This Rd worth $10,000,000. ',
 "Edward's residence is owned by Landlord 227. ",
 "Landlord 227's house was inspected 60 days after the event and suffered $200,000 of damage.",
 'Landlord 227 submitted an insurance claim 60 days after the event. ',
 'Landlord 227 received a $1 insurance payout 75 days after the event. ',
 'Landlord 227 submitted a request to FEMA 75 days after the event. ',
 'Landlord 227 received $199,999 from FEMA 95 days after the event. ',
 'It took Landlord 227 35 days to receive enough financial assistance and now has $200,000 to rebuild. ',
 'Landlord 227 received an engineering assessment 120 days after the event. ',
 'Landlord 227 received permit approval 155 days after the event. ',
 "Landlord 227's home was repaired 157 days after the event, taking 2 days to repair. "]

In [321]:
renters[2].story + renters[2].landlord.story

['Oswald rents and lives in a 0 bedroom Multi Family Dwelling at 2301 Other Ave worth $10,000. ',
 "Oswald's residence is owned by Landlord 232. ",
 "Landlord 232's house was inspected 60 days after the event and suffered $10,000 of damage.",
 'Landlord 232 submitted an insurance claim 60 days after the event. ',
 'Landlord 232 received a $1 insurance payout 75 days after the event. ',
 'Landlord 232 submitted a request to FEMA 75 days after the event. ',
 'Landlord 232 received $9,999 from FEMA 95 days after the event. ',
 'It took Landlord 232 35 days to receive enough financial assistance and now has $10,000 to rebuild. ',
 'Landlord 232 started searching for a Multi Family Dwelling with a value under $10,000 95 days after the event. ',
 'On day 645, after a 550 day search, Landlord 232 gave up looking for a new home in the local area. ']

In [322]:
renters[3].story + renters[3].landlord.story

['James rents and lives in a 2 bedroom Single Family Dwelling at 74000 Out There Lane worth $800,000. ',
 "James's residence is owned by Landlord 236. ",
 "Landlord 236's house was inspected 60 days after the event and suffered $80,000 of damage.",
 'Landlord 236 submitted an insurance claim 60 days after the event. ',
 'Landlord 236 received a $1 insurance payout 75 days after the event. ',
 'Landlord 236 submitted a request to FEMA 75 days after the event. ',
 'Landlord 236 received $79,999 from FEMA 95 days after the event. ',
 'It took Landlord 236 35 days to receive enough financial assistance and now has $80,000 to rebuild. ',
 'Landlord 236 received an engineering assessment 120 days after the event. ',
 'Landlord 236 received permit approval 155 days after the event. ',
 "Landlord 236's home was repaired 185 days after the event, taking 30 days to repair. "]

In [323]:
if not renters[3].residence: print('Displaced')